In [32]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

### Loading Data

In [33]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Getting dummy variables
y_train_fixed = np.zeros((y_train.shape[0], 10))
y_test_fixed = np.zeros((y_test.shape[0], 10))

i = 0
for [val] in y_train:
  y_train_fixed[i][val] = 1
  i += 1
  
i = 0
for [val] in y_test:
  y_test_fixed[i][val] = 1
  i += 1

### Setting up the model

*I found such example on stackExchange:*

```py
model = Sequential()
model.add(Conv2D(input_shape=trainX[0,:,:,:].shape, filters=32, 
                 use_bias=True, kernel_size=(3,3)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Conv2D(filters=64, use_bias=False, kernel_size=(5,5), strides=2))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(n_classes, activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
```

In [34]:
num_classes = 10

model = keras.Sequential(
  [
    keras.Input(shape=(32, 32, 3)),
    layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation="relu"),
    layers.Dropout(0.1),
    layers.Conv2D(32, kernel_size=(3, 3), use_bias=False, padding='same', strides=2, activation="relu"),
    layers.Dropout(0.2),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation="softmax"),
  ]
)

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 dropout_17 (Dropout)        (None, 32, 32, 32)        0         
                                                                 
 conv2d_15 (Conv2D)          (None, 16, 16, 32)        9216      
                                                                 
 dropout_18 (Dropout)        (None, 16, 16, 32)        0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 8, 8, 32)         0         
 2D)                                                             
                                                                 
 flatten_6 (Flatten)         (None, 2048)              0         
                                                      

In [35]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

### Training model

In [36]:
batch_size = 256 # Training data will be split in chunks of this size that will be passed through the network at the same time
epochs = 25 # During one epoch the model 'sees' all of the training data, with this parameter we determine how many times the model will be allowed to see them
validation_split = 0.1 # Determines what percentage of training data will be used to validate the model after each epoch
history = model.fit(x_train, y_train_fixed, batch_size=batch_size, epochs=epochs, validation_split=validation_split)

Epoch 1/25
176/176 [==============================] - 15s 81ms/step - loss: 4.8832 - accuracy: 0.1327 - val_loss: 2.1678 - val_accuracy: 0.1788
Epoch 2/25
176/176 [==============================] - 15s 86ms/step - loss: 2.1432 - accuracy: 0.1722 - val_loss: 1.8867 - val_accuracy: 0.2814
Epoch 3/25
176/176 [==============================] - 15s 88ms/step - loss: 1.9027 - accuracy: 0.2708 - val_loss: 1.6557 - val_accuracy: 0.3818
Epoch 4/25
176/176 [==============================] - 15s 86ms/step - loss: 1.7475 - accuracy: 0.3401 - val_loss: 1.5809 - val_accuracy: 0.4400
Epoch 5/25
176/176 [==============================] - 15s 86ms/step - loss: 1.6657 - accuracy: 0.3802 - val_loss: 1.4882 - val_accuracy: 0.4826
Epoch 6/25
176/176 [==============================] - 15s 86ms/step - loss: 1.5890 - accuracy: 0.4114 - val_loss: 1.4024 - val_accuracy: 0.5164
Epoch 7/25
176/176 [==============================] - 15s 86ms/step - loss: 1.5175 - accuracy: 0.4422 - val_loss: 1.3310 - val_accuracy:

In [37]:
# Additional training
history = model.fit(x_train, y_train_fixed, batch_size=batch_size, epochs=100, validation_split=validation_split)

Epoch 1/100
176/176 [==============================] - 14s 79ms/step - loss: 1.0001 - accuracy: 0.6438 - val_loss: 0.9946 - val_accuracy: 0.6670
Epoch 2/100
176/176 [==============================] - 15s 83ms/step - loss: 0.9940 - accuracy: 0.6495 - val_loss: 0.9879 - val_accuracy: 0.6666
Epoch 3/100
176/176 [==============================] - 15s 86ms/step - loss: 0.9720 - accuracy: 0.6546 - val_loss: 0.9861 - val_accuracy: 0.6652
Epoch 4/100
176/176 [==============================] - 15s 86ms/step - loss: 0.9601 - accuracy: 0.6604 - val_loss: 0.9776 - val_accuracy: 0.6790
Epoch 5/100
176/176 [==============================] - 15s 86ms/step - loss: 0.9543 - accuracy: 0.6624 - val_loss: 0.9739 - val_accuracy: 0.6706
Epoch 6/100
176/176 [==============================] - 15s 86ms/step - loss: 0.9349 - accuracy: 0.6705 - val_loss: 0.9670 - val_accuracy: 0.6710
Epoch 7/100
176/176 [==============================] - 15s 86ms/step - loss: 0.9206 - accuracy: 0.6743 - val_loss: 0.9605 - val_ac